In [6]:
# %%

from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecEnv, VecFrameStack, VecNormalize, VecTransposeImage
from stable_baselines3.common.env_util import make_vec_env
import pybullet_envs

import numpy as np
import torch
import torch.nn as nn
import os
import pickle


import sys
zoo_path = "/home/sgillen/work/external/rl-baselines3-zoo"
sys.path.append(zoo_path + "/utils/")

from utils import ALGOS, get_latest_run_id, get_saved_hyperparams
from utils import get_wrapper_class


env_id = 'Walker2DBulletEnv-v0'
algo = 'ppo'
folder = zoo_path + "/rl-trained-agents/"

exp_id = get_latest_run_id(os.path.join(folder, algo), env_id)
log_path = os.path.join(folder, algo, f"{env_id}_{exp_id}")
assert os.path.isdir(log_path), f"The {log_path} folder was not found"


model_path = os.path.join(log_path, f"{env_id}.zip")
print(model_path)

stats_path = os.path.join(log_path, env_id)
norm_reward = True  ## ???
hyperparams, stats_path = get_saved_hyperparams(stats_path, norm_reward=norm_reward, test_mode=False)


# or kwargs_key in {"policy_kwargs", "replay_buffer_class", "replay_buffer_kwargs"}:
#    if kwargs_key in hyperparams.keys() and isinstance(hyperparams[kwargs_key], str):
#        hyperparams[kwargs_key] = eval(hyperparams[kwargs_key])


env_wrapper = get_wrapper_class(hyperparams)

normalize_path = os.path.join(os.path.dirname(model_path), env_id)
normalize_path = os.path.join(normalize_path, "vecnormalize.pkl")

env = make_vec_env(env_id, wrapper_class=env_wrapper, vec_env_cls=DummyVecEnv)
env = VecNormalize.load(normalize_path, env)

# %%
model = ALGOS[algo].load(model_path, env=env)

/home/sgillen/work/external/rl-baselines3-zoo/rl-trained-agents/ppo/Walker2DBulletEnv-v0_1/Walker2DBulletEnv-v0.zip


In [5]:
print(env)

o_list = []

done = False

o = env.reset()
R = 0
import time
while(not done):
    o_list.append(o)
    
    act = model.predict(o)[0]
    o,r,done,_ = env.step(act)
#    time.sleep(0.01)
    R+=r

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
print(r)
plt.plot(np.array(o_list).squeeze())
plt.show()
    

/home/sgillen/work/external/rl-baselines3-zoo/rl-trained-agents/ppo/Walker2DBulletEnv-v0_1/Walker2DBulletEnv-v0.zip


/home/sgillen/anaconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


AttributeError: 'PPO' object has no attribute 'parameters'

In [6]:
P = list(model.policy.parameters())

In [11]:
model.policy.action_net.parameters()

<generator object Module.parameters at 0x7fab1c19f1a8>